# 🔐 Tenuo: Capability Tokens for AI Agents

**Think prepaid debit card, not corporate Amex.** Tenuo provides ephemeral, scoped capability tokens that expire when the task ends.

This notebook demonstrates:
1. **Basic Flow** - Issue → Attenuate → Authorize
2. **Monotonic Delegation** - Authority can only shrink
3. **BoundWarrant** - Convenient key binding
4. **Debugging** - Why was I denied?
5. **Multi-Mission Pattern** - Temporal least-privilege
6. **Task Scoping** - Orchestrator → Worker pattern

📘 **Next:** [Framework Integrations](https://colab.research.google.com/github/tenuo-ai/tenuo/blob/main/notebooks/tenuo_integrations.ipynb) (LangChain, LangGraph, FastAPI, Google ADK, OpenAI)

[GitHub](https://github.com/tenuo-ai/tenuo) | [Docs](https://tenuo.ai)


In [ ]:
# Install Tenuo (latest version)
%pip install -q tenuo


## 1. Basic Flow: Issue → Attenuate → Authorize

The core Tenuo workflow in 30 seconds.


In [ ]:
from tenuo import SigningKey, Warrant, Pattern, Range

# 1. Generate keys
control_key = SigningKey.generate()
worker_key = SigningKey.generate()

print("Keys generated:")
print(f"  Control: {control_key.public_key}")
print(f"  Worker:  {worker_key.public_key}")


In [ ]:
# 2. Issue a root warrant with constraints
root_warrant = (Warrant.mint_builder()
    .tool("search")
    .tool("calculator")
    .capability("search", query=Pattern("*"))  # Any query
    .capability("calculator", max_result=Range(max=1000))
    .holder(control_key.public_key)
    .ttl(3600)
    .mint(control_key)
)

print("Root warrant issued:")
print(f"  ID: {root_warrant.id}")
print(f"  Tools: {root_warrant.tools}")
print(f"  TTL: {root_warrant.ttl}")


In [ ]:
# 3. Attenuate (delegate with narrower scope)
worker_warrant = (root_warrant.grant_builder()
    .capability("search", query=Pattern("weather *"))  # Narrower: only weather queries
    .holder(worker_key.public_key)
    .ttl(300)
    .grant(control_key)
)

print("Delegated warrant:")
print(f"  ID: {worker_warrant.id}")
print(f"  Depth: {worker_warrant.depth} (attenuated)")
print(f"  TTL: {worker_warrant.ttl}")


In [ ]:
# 4. Authorize requests (with Proof-of-Possession)
import time

def check(warrant, tool, args, key):
    sig = warrant.sign(key, tool, args, int(time.time()))
    return warrant.authorize(tool, args, bytes(sig))

tests = [
    ("search", {"query": "weather NYC"}, True, "matches 'weather *'"),
    ("search", {"query": "weather forecast"}, True, "matches 'weather *'"),
    ("search", {"query": "stock prices"}, False, "doesn't match 'weather *'"),
    ("calculator", {"max_result": 500}, False, "tool not delegated"),
]

print("Authorization tests:")
for tool, args, expected, desc in tests:
    result = check(worker_warrant, tool, args, worker_key)
    status = "✓" if result == expected else "✗"
    print(f"  {status} {tool}({list(args.values())[0]}) → {'ALLOWED' if result else 'BLOCKED'} ({desc})")


## 2. Monotonic Delegation: Authority Can Only Shrink

This is the core security property. Delegated warrants **cannot** add tools or widen constraints.


In [ ]:
from tenuo import SigningKey, Warrant, Pattern

root_key = SigningKey.generate()
child_key = SigningKey.generate()

# Root: broad authority (3 tools, wide path)
root = (Warrant.mint_builder()
    .tool("read_file").tool("write_file").tool("delete_file")
    .capability("read_file", path=Pattern("/data/*"))
    .capability("write_file", path=Pattern("/data/*"))
    .capability("delete_file", path=Pattern("/data/*"))
    .holder(root_key.public_key)
    .ttl(3600)
    .mint(root_key)
)
print(f"Root: {root.tools}, path=/data/*")

# Child: drops delete, narrows path
child = (root.grant_builder()
    .capability("read_file", path=Pattern("/data/reports/*"))
    .capability("write_file", path=Pattern("/data/reports/*"))
    .holder(child_key.public_key)
    .grant(root_key)
)
print(f"Child: {child.tools}, path=/data/reports/*")


In [ ]:
# Try to RE-ADD delete_file → BLOCKED
print("Attempting to re-add 'delete_file'...")
try:
    grandchild_key = SigningKey.generate()
    evil = (child.grant_builder()
        .capability("delete_file", path=Pattern("/data/*"))  # Escalation!
        .holder(grandchild_key.public_key)
        .grant(child_key)
    )
    print("❌ SECURITY FAILURE!")
except Exception as e:
    print(f"✓ Blocked: {type(e).__name__}")

# Try to WIDEN path → BLOCKED  
print("\nAttempting to widen path...")
try:
    evil = (child.grant_builder()
        .capability("read_file", path=Pattern("/data/*"))  # Widening!
        .holder(grandchild_key.public_key)
        .grant(child_key)
    )
    print("❌ SECURITY FAILURE!")
except Exception as e:
    print(f"✓ Blocked: {type(e).__name__}")

print("\n🔐 Monotonicity enforced: delegation can only REDUCE authority")


## 3. BoundWarrant: Convenient Key Binding

Bind a warrant to its key once, use it repeatedly without passing the key.


In [ ]:
from tenuo import SigningKey, Warrant, Pattern

key = SigningKey.generate()
warrant = (Warrant.mint_builder()
    .tool("search")
    .capability("search", query=Pattern("*"))
    .holder(key.public_key)
    .ttl(300)
    .mint(key)
)

# Bind key to warrant
bound = warrant.bind(key)

print("BoundWarrant:")
print(f"  ID: {bound.warrant.id}")
print(f"  TTL: {bound.ttl}")
print(f"  Expired: {bound.is_expired}")


In [ ]:
# Authorize without passing key each time
result = bound.validate("search", {"query": "test"})
print(f"Authorized: {result}")

# Generate HTTP headers for API calls
headers = bound.headers("search", {"query": "test"})
print("\nHTTP Headers for API calls:")
for k, v in headers.items():
    print(f"  {k}: {v[:40]}...")


## 4. Debugging: Why Was I Denied?

Get structured explanations for authorization failures.


In [ ]:
from tenuo import SigningKey, Warrant, Pattern, Range

key = SigningKey.generate()
warrant = (Warrant.mint_builder()
    .tool("search")
    .capability("search", query=Pattern("weather *"))
    .holder(key.public_key)
    .ttl(300)
    .mint(key)
)

# Check why requests are denied
test_cases = [
    ("search", {"query": "stock prices"}),
    ("delete_file", {"path": "/etc/passwd"}),
]

print("Denial explanations:\n")
for tool, args in test_cases:
    result = warrant.why_denied(tool, args)
    print(f"  {tool}({args})")
    print(f"    Denied: {result.denied}")
    print(f"    Code: {result.deny_code}")
    if result.suggestion:
        print(f"    Suggestion: {result.suggestion}")
    print()


## 5. Multi-Mission Pattern: Temporal Least-Privilege

Real agents don't have one job—they handle multiple missions over time. Each mission gets exactly the authority it needs, **no more**.

**Key insight:** Same agent, same session, but different warrants for different tasks. If one warrant is compromised, the attacker can't pivot to other missions.



In [ ]:
from tenuo import SigningKey, Warrant, Pattern, Range

# Orchestrator and worker keys (same agent, same session)
orchestrator = SigningKey.generate()
worker = SigningKey.generate()

# Issue mission-scoped warrants from orchestrator to worker
print("📋 Orchestrator issuing mission-specific warrants:\n")

# MISSION A: File Operations (read-only, limited paths)
mission_a = (Warrant.mint_builder()
    .tool("read_file")
    .capability("read_file", path=Pattern("/data/*"))
    .holder(worker.public_key)
    .ttl(300)  # 5 minutes
    .mint(orchestrator)
)
print("   📁 Mission A: read_file, path=/data/*, TTL=5min")

# MISSION B: Infrastructure (cluster management)
# Note: Glob uses {a,b} for alternation, not a|b
mission_b = (Warrant.mint_builder()
    .tool("manage_infrastructure")
    .capability("manage_infrastructure", 
                cluster=Pattern("staging-*"),
                action=Pattern("{upgrade,restart}"),
                replicas=Range(max=10))
    .holder(worker.public_key)
    .ttl(600)  # 10 minutes
    .mint(orchestrator)
)
print("   🔧 Mission B: manage_infrastructure, cluster=staging-*, TTL=10min")

print(f"\n✓ Same worker ({str(worker.public_key)[:16]}...), different capabilities")



In [ ]:
# Test 1: Mission A warrant for file operations → SUCCESS
print("📁 Using Mission A warrant:\n")

bound_a = mission_a.bind(worker)
result = bound_a.validate("read_file", {"path": "/data/config.json"})
print(f"   read_file('/data/config.json') → {'✅ ALLOWED' if result else '❌ DENIED'}")

# Test 2: Mission A warrant for infrastructure → DENIED (wrong mission!)
result = bound_a.validate("manage_infrastructure", {"cluster": "staging-web", "action": "upgrade"})
print(f"   manage_infrastructure('staging-web') → {'❌ DENIED' if not result else '⚠️ ALLOWED (unexpected)'}")
print("   → Mission A warrant cannot do infrastructure operations\n")

# Test 3: Mission B warrant for infrastructure → SUCCESS  
print("🔧 Using Mission B warrant:\n")

bound_b = mission_b.bind(worker)
result = bound_b.validate("manage_infrastructure", {"cluster": "staging-web", "action": "upgrade", "replicas": 5})
print(f"   manage_infrastructure('staging-web', 'upgrade') → {'✅ ALLOWED' if result else '❌ DENIED'}")

# Test 4: Mission B warrant for file operations → DENIED (wrong mission!)
result = bound_b.validate("read_file", {"path": "/data/config.json"})
print(f"   read_file('/data/config.json') → {'❌ DENIED' if not result else '⚠️ ALLOWED (unexpected)'}")
print("   → Mission B warrant cannot do file operations")



In [ ]:
print("─" * 60)
print("\n🔐 KEY INSIGHT: Temporal Least-Privilege")
print("─" * 60)
print("""
Same worker, same session, but warrants are mission-scoped.

• Mission A (files):        5 min TTL, /data/* only
• Mission B (infrastructure): 10 min TTL, staging-* only

If an attacker compromises Mission A warrant:
  ✗ Cannot access infrastructure (wrong tool)
  ✗ Cannot access /etc/* (path constraint)
  ✗ Expires in 5 minutes anyway

This is the "prepaid debit card" model:
  → Each mission gets exactly what it needs
  → No lingering authority after task completion
  → Blast radius is contained to one mission
""")



In [ ]:
# Quick status checks (using Mission A warrant from above)
print("Mission A Warrant status:")
print(f"  ID: {mission_a.id}")
print(f"  TTL: {mission_a.ttl}")
print(f"  Expires at: {mission_a.expires_at()}")
print(f"  Is expired: {mission_a.is_expired()}")
print(f"  Is terminal: {mission_a.is_terminal()}")
print(f"  Tools: {mission_a.tools}")


## 6. Task Scoping: Orchestrator → Worker Pattern

The Tier 1 API for quick prototyping. Each worker gets exactly the authority it needs.


In [ ]:
from tenuo import configure, mint, grant, guard
from tenuo import SigningKey, Pattern, Capability

# Configure (dev mode for prototyping)
configure(issuer_key=SigningKey.generate(), dev_mode=True, audit_log=False)

# Define tools with @guard decorator
@guard
async def search_web(query: str) -> str:
    return f"🔍 Results: {query}"

@guard
async def read_file(path: str) -> str:
    return f"📄 Read: {path}"

@guard
async def send_email(to: str, body: str) -> str:
    return f"📧 Sent to {to}"

search, read, email = search_web, read_file, send_email

print("Tools protected ✓")


In [ ]:
async def demo():
    print("=== Orchestrator → Worker Pattern ===\n")
    
    # Orchestrator scope
    # Note: _allow_unknown=True allows unconstrained fields (like 'body' for email)
    # This is needed because Tenuo uses zero-trust mode when ANY constraint is defined
    async with mint(
        Capability("search_web", query=Pattern("*")),
        Capability("read_file", path=Pattern("/data/*")),
        Capability("send_email", to=Pattern("*@company.com"), _allow_unknown=True),
    ):
        # Worker 1: Research (search only)
        print("🔧 Worker 1: Research")
        async with grant(Capability("search_web", query=Pattern("*"))):
            print(f"   {await search(query='Q3 data')}")
            try:
                await read(path="/data/secret.txt")
            except Exception as e:
                print(f"   ✓ Cannot read: {type(e).__name__}")
        
        # Worker 2: Report (read + email)
        print("\n🔧 Worker 2: Report")
        async with grant(
            Capability("read_file", path=Pattern("/data/reports/*")),
            Capability("send_email", to=Pattern("*@company.com"), _allow_unknown=True),
        ):
            print(f"   {await read(path='/data/reports/q3.txt')}")
            print(f"   {await email(to='cfo@company.com', body='Report')}")
            try:
                await email(to="leak@evil.com", body="secrets")
            except Exception as e:
                print(f"   ✓ Cannot email external: {type(e).__name__}")
    
    print("\n🔐 Each worker had exactly the authority needed.")

await demo()


---

## Next Steps

### 🔌 [Framework Integrations Notebook](https://colab.research.google.com/github/tenuo-ai/tenuo/blob/main/notebooks/tenuo_integrations.ipynb)

See Tenuo in action with LangChain, LangGraph, FastAPI, Google ADK, and OpenAI.

**Quick imports:**
- **LangChain**: `from tenuo.langchain import guard`
- **LangGraph**: `from tenuo.langgraph import TenuoToolNode`
- **FastAPI**: `from tenuo.fastapi import TenuoGuard`
- **Templates**: `from tenuo.templates import FileReader, WebSearcher, CommonAgents`
- **OpenAI**: `from tenuo.openai import GuardBuilder, Subpath, UrlSafe`
- **Google ADK**: `from tenuo.google_adk import GuardBuilder, protect_agent`

**Pre-built Templates** (see Integrations notebook):
```python
from tenuo.templates import FileReader, DatabaseReader, CommonAgents

# Instead of: Capability("read_file", path=Pattern("/data/*"))
# Use: FileReader.in_directory("/data")

# Pre-built agent patterns:
research_agent = CommonAgents.research_assistant(
    search_domains=["arxiv.org"],
    output_dir="/tmp/research"
)
```

**Resources:**
- 📦 [GitHub](https://github.com/tenuo-ai/tenuo)
- 📖 [Documentation](https://tenuo.ai)
- 💡 [Examples](https://github.com/tenuo-ai/tenuo/tree/main/tenuo-python/examples) (including **AgentQL** & **MCP**)

**Install:**
```bash
uv pip install tenuo                  # Core
uv pip install "tenuo[langchain]"     # + LangChain
uv pip install "tenuo[langgraph]"     # + LangGraph
uv pip install "tenuo[fastapi]"       # + FastAPI
```

---

### 🔍 [Tenuo Explorer](https://tenuo.ai/explorer/)

Decode, build, and test warrants visually in your browser. Paste any warrant to see its structure, test authorization, or generate Python/Rust code.
